In [1]:
import spacy
from spacy.tokens import Doc
from spacy.training import Example
from datasets import load_dataset
from tqdm import tqdm
import random
import os
import numpy as np
from spacy.util import minibatch

dataset = load_dataset("conll2003")
print("Dataset structure:", dataset)

label_list = dataset["train"].features["ner_tags"].feature.names
print("Labels:", label_list)

epochs = 35
max_samples = None
batch_size = 32


# Create a spaCy Doc with the tokens
def create_training_examples(nlp, dataset_split, max_samples=None):
    examples = []
    
    sample_count = len(dataset_split) if max_samples is None else min(max_samples, len(dataset_split))
    
    for i in tqdm(range(sample_count), desc=f"Processing {sample_count} examples"):
        item = dataset_split[i]
        tokens = item["tokens"]
        ner_tags = item["ner_tags"]
        
        spaces = [True] * len(tokens)
        if spaces:  
            spaces[-1] = False
        
        doc = Doc(nlp.vocab, words=tokens, spaces=spaces)
        entities = []
        current_entity = None
        
        for token_idx, (token, tag_id) in enumerate(zip(doc, ner_tags)):
            tag = label_list[tag_id]
            
            if tag.startswith("B-") or tag.startswith("I-") or tag == "O":
                if current_entity is not None:
                    # Add the previous entity
                    entities.append(current_entity)
                    current_entity = None
                # Start a new entity
                current_entity = {
                    "start": token.idx,
                    "end": token.idx + len(token.text),
                    "label": tag 
                }
        
        if current_entity is not None:
            entities.append(current_entity)
        
        reference = doc.copy()
        for ent in entities:
            span = reference.char_span(ent["start"], ent["end"], label=ent["label"])
            if span is not None:
                reference.ents = list(reference.ents) + [span]
        
        # Create example
        example = Example(doc, reference)
        examples.append(example)
    return examples


/Users/anujsolanki/nlp-assignment/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset structure: DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})
Labels: ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']


In [2]:

def train_spacy_model(train_examples):
    nlp = spacy.blank("en")
    
    if "ner" not in nlp.pipe_names:
        ner = nlp.add_pipe("ner", last=True)
    else:
        ner = nlp.get_pipe("ner")
    
    unique_labels = set()
    for label in label_list:
        unique_labels.add(label)
    
    for label in unique_labels:
        print(f"Adding label: {label}")
        ner.add_label(label)
    
    n_iter = epochs
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    
    training_losses = []
    
    print(f"Training with {len(train_examples)} examples")
    
    # Train the model
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()
        
        for i in range(n_iter):
            random.shuffle(train_examples)
            
            losses = {}
            batches = minibatch(train_examples, size=batch_size)
            
            for batch in tqdm(list(batches), desc=f"Epoch {i+1}/{n_iter}"):
                nlp.update(batch, drop=0.2, losses=losses)
            
            epoch_loss = losses.get("ner", 0)
            training_losses.append(epoch_loss)
            print(f"Epoch {i+1}/{n_iter}, Loss: {epoch_loss:.4f}")
        
    
    return nlp, {"training_losses": training_losses}


In [3]:

nlp = spacy.blank("en")

# Create training and validation data
print("Preparing training data...")
train_examples = create_training_examples(nlp, dataset["train"], max_samples=max_samples) 


Preparing training data...


Processing 14041 examples: 100%|██████████| 14041/14041 [00:05<00:00, 2763.68it/s]


In [4]:

# Train model
print("Training model...")
trained_model, metrics = train_spacy_model(train_examples)



Training model...
Adding label: B-ORG
Adding label: I-ORG
Adding label: B-MISC
Adding label: O
Adding label: B-LOC
Adding label: I-PER
Adding label: I-MISC
Adding label: I-LOC
Adding label: B-PER
Training with 14041 examples


Epoch 1/35: 100%|██████████| 439/439 [00:41<00:00, 10.58it/s]


Epoch 1/35, Loss: 24167.9258


Epoch 2/35: 100%|██████████| 439/439 [00:42<00:00, 10.41it/s]


Epoch 2/35, Loss: 9244.8447


...

Epoch 33/35, Loss: 727.5371


Epoch 34/35: 100%|██████████| 439/439 [00:40<00:00, 10.81it/s]


Epoch 34/35, Loss: 664.5427


Epoch 35/35: 100%|██████████| 439/439 [00:41<00:00, 10.62it/s]

Epoch 35/35, Loss: 708.0412


In [5]:
# Save model to pickle
import pickle
with open("spacy_ner_model.pkl", "wb") as f:
    pickle.dump(trained_model, f)

In [15]:
def predict(model, text):
    doc = model(text)
    token_entities = []
    token = []
    for ent in doc.ents:
        token.append(ent.text)
        token_entities.append(ent.label_)
    return token,token_entities

text = "Apple Inc is an American multinational technology company headquartered in Cupertino, California."
tokens,entities = predict(trained_model, text)
print(tokens)
print(entities)

dataset = load_dataset("conll2003")
test_examples = dataset["test"][:500]



['Apple', 'Inc', 'is', 'an', 'American', 'multinational', 'technology', 'company', 'headquartered', 'in', 'Cupertino', ',', 'California', '.']
['B-ORG', 'I-ORG', 'O', 'O', 'B-MISC', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'B-LOC', 'O']


In [14]:
from collections import Counter

def calculate_ner_metrics(true_entities, pred_entities, labels):
    assert len(true_entities) == len(pred_entities), "Mismatch in number of sentences"

    # Flatten lists
    y_true = [label for seq in true_entities for label in seq]
    y_pred = [label for seq in pred_entities for label in seq]

    # Calculate TP, FP, FN for each label
    label_counts = {label: Counter() for label in labels}

    for true, pred in zip(y_true, y_pred):
        if true == pred:
            label_counts[true]['TP'] += 1
        else:
            label_counts[true]['FN'] += 1
            label_counts[pred]['FP'] += 1

    # Compute metrics for each label
    metrics = {}
    total_tp = total_fp = total_fn = 0

    for label, counts in label_counts.items():
        tp, fp, fn = counts['TP'], counts['FP'], counts['FN']
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        total_tp += tp
        total_fp += fp
        total_fn += fn

        metrics[label] = {"Precision": precision, "Recall": recall, "F1-score": f1}

    # Micro-averaged scores
    overall_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    overall_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
    overall_f1 = 2 * (overall_precision * overall_recall) / (overall_precision + overall_recall) if (overall_precision + overall_recall) > 0 else 0

    # Accuracy
    accuracy = sum(1 for true, pred in zip(y_true, y_pred) if true == pred) / len(y_true)

    metrics["Overall"] = {
        "Precision": overall_precision,
        "Recall": overall_recall,
        "F1-score": overall_f1,
        "Accuracy": accuracy
    }

    return metrics


def evaluate(model, test_dataset, label_list):

    
    true_entities = []
    pred_entities = []
    tokens = test_dataset["tokens"]
    ner_tags = test_dataset["ner_tags"]
    x = 5
    for i in range(len(tokens)):
        token = tokens[i]
        tag = ner_tags[i]
        tag_with_label = [label_list[tag_id] for tag_id in tag]
        text = " ".join(token)
        _, predicted_entities = predict(model, text)
        true_entities.append(tag_with_label)
        pred_entities.append(predicted_entities)
    
    metrics = calculate_ner_metrics(true_entities, pred_entities, label_list)
    print("NER Metrics:")
    for label, scores in metrics.items():
        print(f"Label: {label}")
        for metric, score in scores.items():
            print(f"{metric}: {score:.4f}")
        print()
    print(len(metrics))
    return metrics


import pickle
with open("spacy_ner_model.pkl", "rb") as f:
    model = pickle.load(f)

test_dataset = test_examples
label_list = dataset["train"].features["ner_tags"].feature.names
print("Labels:", label_list)
metric = evaluate(model, test_dataset,label_list=label_list)   
    

Labels: ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']
NER Metrics:
Label: O
Precision: 0.8614
Recall: 0.8657
F1-score: 0.8635

Label: B-PER
Precision: 0.1667
Recall: 0.1667
F1-score: 0.1667

Label: I-PER
Precision: 0.1667
Recall: 0.1667
F1-score: 0.1667

Label: B-ORG
Precision: 0.1250
Recall: 0.1429
F1-score: 0.1333

Label: I-ORG
Precision: 0.0000
Recall: 0.0000
F1-score: 0.0000

Label: B-LOC
Precision: 0.0000
Recall: 0.0000
F1-score: 0.0000

Label: I-LOC
Precision: 0.0000
Recall: 0.0000
F1-score: 0.0000

Label: B-MISC
Precision: 0.3333
Recall: 0.2857
F1-score: 0.3077

Label: I-MISC
Precision: 0.0000
Recall: 0.0000
F1-score: 0.0000

Label: Overall
Precision: 0.7553
Recall: 0.7553
F1-score: 0.7553
Accuracy: 0.7553

10
